In [1]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler, MinMaxScaler, ChiSqSelector
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.types import *

In [2]:
try:
  from pyspark.sql import SparkSession
except:
  import findspark
  findspark.init()
  from pyspark.sql import SparkSession

  spark = SparkSession.builder.appName("COVID") \
          .config("hive.exec.dynamic.partition", "true") \
          .config("hive.exec.dynamic.partition.mode", "nonstrict")\
          .enableHiveSupport()\
          .getOrCreate()

In [3]:
def generaDataset(fileloc, columnas):
  """
  fileloc - string que da la dirección del csv con nuestros datos
  columnas - lista con las columnas que queremos utilizar en el dataframe
  
  Devuelve un dataframe a partir del csv en fileloc con las columnas requeridas
  """
  df = spark.read.csv(fileloc, sep = ",", header = True, encoding = "UTF-8", inferSchema = True)
  df = dataClean(df, columnas)
  print("Se generó el dataframe:")
  print(df.show(5))
  print("con %d registros" % df.count())
  return df

def dataClean(df, columnas):
  """
  Filtra el dataframe df  por positivos a Covid
  """
  df = df.filter(df["RESULTADO"] == "Positivo SARS-CoV-2")
  df = df.select(columnas)
  return df

In [4]:
#columnas a utilizar
columnas = ["SEXO", "OBESIDAD", "DIABETES", "EPOC", "ASMA", "CARDIOVASCULAR", "RENAL CRONICA", "TABAQUISMO", "TIPO PACIENTE", "EDAD"]
file = "/FileStore/tables/casos_asociados_a_covid_19-4388c.csv"

df = generaDataset(file, columnas)
train, test = df.randomSplit([0.70, 0.30])

Se generó el dataframe:
+------+--------+--------+----+----+--------------+-------------+----------+-------------+----+
 SEXO|OBESIDAD|DIABETES|EPOC|ASMA|CARDIOVASCULAR|RENAL CRONICA|TABAQUISMO|TIPO PACIENTE|EDAD|
+------+--------+--------+----+----+--------------+-------------+----------+-------------+----+
 MUJER| SI| SI| NO| NO| NO| NO| NO| AMBULATORIO| 61|
HOMBRE| NO| NO| NO| NO| NO| NO| NO| AMBULATORIO| 54|
HOMBRE| NO| NO| NO| NO| NO| NO| NO| AMBULATORIO| 30|
 MUJER| NO| NO| NO| NO| NO| NO| NO|HOSPITALIZADO| 51|
 MUJER| NO| NO| NO| SI| NO| NO| NO| AMBULATORIO| 32|
+------+--------+--------+----+----+--------------+-------------+----------+-------------+----+
only showing top 5 rows

None
con 6296 registros

In [5]:
df.groupBy("TIPO PACIENTE").count().show()

+-------------+-----+
TIPO PACIENTE|count|
+-------------+-----+
 AMBULATORIO| 4030|
HOSPITALIZADO| 2266|
+-------------+-----+

In [6]:
def pruebaChiSquare(data, numCols, catCols, target):
  """
  Realiza la prueba de chi^2 en el dataset provisto después de un poco de procesamiento 
  Regresa el dataset procesado y con sólo con las variables relevantes, a partir de la prueba de chi^2
  
  data -- dataframe a utilizar
  numCols -- lista con las features numéricas del dataset
  catCols -- lista con las features categóricas del dataset
  target -- string con el nombre la variable objetivo a utilizar para la prueba
  """
  stages = []
  for categoricalCol in catCols:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + "index")
    stages += [stringIndexer]
  labelStringIndexer = StringIndexer(inputCol = target, outputCol = "label")
  stages += [labelStringIndexer]
  assemblerInputs = [c + "index" for c in catCols] + numCols
  assembler = VectorAssembler(inputCols = assemblerInputs, outputs = "features")
  css = ChiSqSelector(features = 'features', outputCol = "salidaf", labelCol = 'label', fpr = 0.05)
  stages += [assembler, css]
  pipe = Pipeline().setStages(stages)
  result = pipe.fit(data).transform(data)
  return result

In [7]:
def entrenamiento(data, catCols, numCols, target):
  """
  Crea un pipeline donde las variables categóricas se procesan usando one-hot-encoding
  pasa la data transformada al modelo logístico y regresa el modelo ajustado.
  
  data -- el dataframe a utilizar
  catCols -- lista con las columnas categóricas de las features a usar
  numCols -- lista con las columnas numéricas de las features
  target -- la variable que deseamos modelar con la regresión logística
  """
  stages = []
  for categoricalCol in catCols:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + "index")
    stages += [stringIndexer]
    ohe = OneHotEncoderEstimator(inputCols = [stringIndexer.getOutputCol()], outputCols = [categoricalCol + "ohe"])
    stages += [ohe]
  label_stringIdx = StringIndexer(inputCol = target, outputCol = "label")
  stages += [label_stringIdx]
  assemblerInputs = [c+"ohe" for c in catCols] + numCols
  assembler = VectorAssembler(inputCols = assemblerInputs, outputCol = "featuresVec")
  minmax = MinMaxScaler(inputCol = "featuresVec", outputCol = "features")
  lr = LogisticRegression(labelCol = "label", featuresCol = "features", maxIter = 10)
  stages += [assembler, minmax, lr]
  pipe = Pipeline().setStages(stages)
  model = pipe.fit(data)
  
  print("Coeficientes: " + str(model.stages[-1].coefficientMatrix))
  print("Intercepto: " + str(model.stages[-1].interceptVector))
  return model

In [8]:
def metricas(prediccion):
  """
  Regresa el accuracy y la curva ROC del modelo 
  
  prediccion -- dataframe con el dataset de test
  """
  score = MulticlassClassificationEvaluator(
  predictionCol = "prediction",
  labelCol = "label",
  metricName = "accuracy")
  
  evaluator = BinaryClassificationEvaluator(rawPredictionCol = "rawPrediction", labelCol = "label")
  print("El accuracy del modelo es:")
  print(score.evaluate(prediccion))
  print("La curva ROC del modelo es:")
  print(evaluator.evaluate(prediccion))

In [9]:
numericalColumns = columnas[-1:]
target = columnas[-2]
categoricalColumns = columnas[:-2]
modelo = entrenamiento(train, categoricalColumns, numericalColumns, target)

Exception ignored in: <function JavaWrapper.__del__ at 0x7f9c1fc1dc80>
Traceback (most recent call last):
 File "/databricks/spark/python/pyspark/ml/wrapper.py", line 40, in __del__
 if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'VectorAssembler' object has no attribute '_java_obj'
Coeficientes: DenseMatrix([[ 0.53528989, -0.30821661, 0.2012714 , -0.42596433, 0.38316161,
 -0.49689643, 0.31850112, -0.07320358, -0.22327416, -0.32468339,
 -0.09725334, -0.78408543, 0.32849561, -0.21189713, 0.06657103,
 4.04393641]])
Intercepto: [-0.5794737210407417]

In [10]:
prediccion = modelo.transform(train)
metricas(prediccion)

El accuracy del modelo es:
0.7149372862029647
La curva ROC del modelo es:
0.7510262424370612

In [11]:
def structDataframe(data):
  """
  Dada una lista con entradas de datos, regresa un dataframe con dichos datos y el esquema de abajo
  data -- lista con listas, cada lista tiene datos de EDAD, SEXO, etc...
  """
  schema = StructType([
    StructField("EDAD", IntegerType(), True),
    StructField("SEXO", StringType(), True),
    StructField("OBESIDAD", StringType(), True),
    StructField("DIABETES", StringType(), True),
    StructField("EPOC", StringType(), True),
    StructField("ASMA", StringType(), True),
    StructField("CARDIOVASCULAR", StringType(), True),
    StructField("TABAQUISMO", StringType(), True),
    StructField("RENAL CRONICA", StringType(), True)
  ])
  
  df = spark.createDataFrame(data, schema)
  return df

In [12]:
data = [[25, "HOMBRE", "NO", "SI", "SI", "NO", "NO", "NO", "SI"], [70, "MUJER", "NO", "SI", "NO", "NO", "NO", "NO", "NO"]]
df2 = structDataframe(data)
result = modelo.transform(df2)